In [ ]:
len(unique_chars) # max length of unique char is 44
maxlen_response # max length of response sequence is 143
maxlen_query # # max length of query sequence is 54

In [ ]:
%pip install --upgrade pip
%pip install numpy
%pip install tensorflow==2.19.0

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
conversations = [
    ("Hi", "Hello! How can I help you?"),
    ("What is your name?", "I am a chatbot."),
    ("How does the weather look today?", "I'm sorry, I don't have information on the weather."),
    ("Goodbye", "Goodbye! Have a great day."),
    ("Can you tell me a joke?", "Why don't skeletons fight each other? They don't have the guts!"),
    ("What is 2 + 2?", "2 + 2 equals 4."),
    ("Tell me a fun fact.", "Did you know that honey never spoils? Archaeologists have found pots of honey in ancient tombs that are over 3,000 years old."),
    ("How are you?", "I'm just a chatbot, but thanks for asking! How are you?"),
    ("Where are you from?", "I don't have a physical location, but I exist here to assist you!"),
    ("What can you do?", "I can help with answering questions, telling jokes, providing facts, and more! Just ask away."),
    ("Can you help me with homework?", "I can certainly try! What subject is it?"),
    ("What is the capital of France?", "The capital of France is Paris."),
    ("Who won the 2020 US Presidential election?", "Joe Biden won the 2020 US Presidential election."),
    ("What time is it?", "I don't have access to real-time data, but you can check the time on your device."),
    ("What's the weather like in New York?", "I’m sorry, I can't provide live weather updates. Please check a weather app for the current conditions."),
    ("Do you like movies?", "I don't have personal preferences, but I can tell you about popular movies if you like!"),
    ("What is your favorite color?", "I don't have preferences, but blue is a popular color. What about you?"),
    ("How old are you?", "I don’t age since I’m a chatbot, but I’m always ready to chat!"),
    ("What is 5 times 6?", "5 times 6 equals 30."),
    ("Can you speak multiple languages?", "Yes, I can assist in many languages! How can I help you in another language?"),
    ("Good morning", "Good morning! How can I assist you today?"),
    ("Are you real?", "I’m real in the sense that I exist as a program, but I don't have physical form."),
    ("What's your purpose?", "My purpose is to assist with information, conversation, and fun interactions!"),
    ("What products do you sell?", "We sell a wide range of products including electronics, textiles, machinery, fruits, and vegetables."),
    ("When are your office hours?", "Our office is open from 9 AM to 5 PM, Monday to Friday."),
    ("Can you tell me more about your export services?", "We specialize in exporting consumer goods, industrial equipment, agricultural products like fruits and vegetables, and much more."),
    ("Do you sell products in different countries?", "Yes, we export to several countries around the world including the USA, Germany, China, India, and many others."),
    ("Which countries do you export to?", "We export to a wide variety of countries including the United States, Canada, the United Kingdom, Australia, and many more."),
    ("Can you sell products to different countries?", "Absolutely! We are capable of handling international shipments to different countries depending on your requirements."),
    ("How do I place an order for export?", "You can place an order by contacting us via our website or directly reaching out to our customer service team."),
    ("Do you offer bulk export services?", "Yes, we offer bulk export services for large orders. Let us know the quantity, and we will provide you with a tailored solution."),
    ("What type of products do you specialize in?", "We specialize in electronics, agricultural products such as fruits and vegetables, textiles, and consumer goods."),
    ("Do you offer custom export packages?", "Yes, we offer custom export packages based on the specific needs and requirements of our clients."),
    ("What is the minimum order quantity?", "The minimum order quantity varies by product. Please contact us for more details about your specific requirements."),
    ("Can I track my order?", "Yes, once your order is dispatched, we provide tracking information so you can follow the shipment."),
    ("What types of payment methods do you accept?", "We accept various payment methods, including bank transfers, credit cards, and online payment systems."),
    ("Are there any export restrictions?", "Certain products may have export restrictions based on international laws. We will inform you if any restrictions apply."),
    ("Do you handle customs clearance?", "Yes, we handle customs clearance for international shipments, ensuring your products are delivered without issues."),
    ("How long does international shipping take?", "International shipping times vary depending on the destination country and the type of product. On average, it can take between 7 to 30 days."),
    ("Can you handle perishable goods?", "Yes, we can arrange the transportation of perishable goods like fruits and vegetables using temperature-controlled containers."),
    ("What is the cost of shipping?", "Shipping costs vary depending on the weight, size, and destination of the products. Please contact us for a quote."),
    ("Do you offer insurance on shipments?", "Yes, we offer insurance on shipments to ensure your goods are protected during transit."),
    ("How can I get in touch with your sales team?", "You can contact our sales team via email or phone, or simply fill out a form on our website."),
    ("Can I visit your office?", "Yes, you're welcome to visit our office during business hours. We would be happy to arrange a meeting."),
    ("What documents are needed for international shipping?", "You will typically need a commercial invoice, packing list, and shipping instructions, among others. We can help guide you through the process."),
    ("Do you handle requests for specific products?", "Yes, we accept requests for specific products. If you're looking for something unique, feel free to let us know!"),
    ("Can I request a product not listed in your catalog?", "Absolutely! If there’s a specific product you're interested in, we can work on sourcing it for you."),
    ("Are you open to new business opportunities?", "Yes, we are always open to new business opportunities and partnerships in the import/export industry."),
    ("Do you collaborate with other businesses?", "Yes, we collaborate with various businesses to expand our reach and provide high-quality products worldwide."),
    ("Can you supply rare or specialized products?", "Yes, we have the capability to source and supply rare or specialized products based on customer demand."),
    ("Can you supply seasonal products?", "Yes, we can supply seasonal products like fruits and vegetables depending on the time of year and availability."),
    ("Are you looking to expand to new markets?", "Yes, we are always looking for opportunities to expand our market presence and work with new partners around the world."),
    ("Do you supply products for importers and distributors?", "Yes, we regularly supply products for importers and distributors who operate in different regions."),
    ("Can you help me with sourcing a new product?", "Yes, we can assist you in sourcing new products based on your requirements and location.")
]

conversations_lower = []
for conv in conversations:
    conv_new=conv[0].lower(),conv[1].lower()
    conversations_lower.append(conv_new)
corpus=""
for conv in conversations_lower:
  for element in conv:
    corpus=corpus+" "+element
unique_chars=sorted(set(corpus))
char_indices=dict()
indices_char=dict()
for number, character in enumerate(unique_chars):
    indices_char[number] = character
    char_indices[character] = number

maxlen_query=max(len(conv[0]) for conv in conversations_lower)
maxlen_response=54 # here we keep the response value same as output which 54
input_data=[]
target_data=[]

for conv in conversations_lower:
  query=conv[0]
  response=conv[1]
  query_converted=[char_indices[char]for char in query]
  response_converted=[char_indices[char]for char in response]

  input_data.append(query_converted)
  target_data.append(response_converted)
input_data=pad_sequences(input_data,maxlen=maxlen_query,padding="post")
target_data =pad_sequences(target_data, maxlen=maxlen_response, padding="post")
embedding_dim=42
model= Sequential()
# here we our unique char is 44,embedding we have set to 42,max query length is 54 this is first layer
model.add(Embedding(len(unique_chars),embedding_dim,input_length=54)) ### here the output is 54
# Second layer will be of RNN which we will add 3 layers of it
model.add(SimpleRNN(54, return_sequences=True))
model.add(SimpleRNN(54, return_sequences=True))
model.add(SimpleRNN(54, return_sequences=True))

#output layer
model.add(Dense(len(unique_chars),activation="softmax"))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(input_data, target_data, epochs=300)
def genrate_response(user_query):
  query_converted=[char_indices[char]for char in user_query]
  query_converted_padded=pad_sequences([query_converted],maxlen=maxlen_query,padding="post")
  predicted_seq=model.predict(query_converted_padded)[0]
  response=""
  for char_probablity in predicted_seq:
    index=np.argmax(char_probablity)
    char=indices_char[index]
    response=response+char
  return response

# for single use:
# user_input=input()
# genrate_response(user_input.lower())


# Continuous loop for user interaction
print("Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ").lower()
    if user_input in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    bot_response = genrate_response(user_input)
    print("Alan:", bot_response)
